# Data Acquisition and Live Inference
**Author:** Enej Podlipnik  
**Affiliation:** University of Ljubljana, Faculty of Mechanical Engineering  
**Date:** July 2025

This notebook is part of the diploma project "Impact Location and Force Prediction."  
For full context, see the repository.

## Purpose
This notebook demonstrates the process of acquiring sensor data and performing live inference using a trained deep learning model.

It is intended for academic and non-commercial use.

## Requirements
- National Instruments hardware with NI-DAQmx drivers
- Python packages listed in requirements.txt

## 1. Device Discovery and Imports

In [1]:
# Imports
import LDAQ
import nidaqmx
from nidaqmx.system import System
import numpy as np
import pathlib
from pathlib import Path
import pickle
import functions
import torch

# Print the devices
system = System.local()
print(f"Devices: {[d.name for d in system.devices]}")


Devices: ['cDAQ1', 'cDAQ1Mod1', 'cDAQ1Mod2', 'cDAQ1Mod3']


## 2. List Available Functions

In [ ]:
functions.list_functions()

## 3. Data Acquisition

In [ ]:
# Trigger forces for the dataset are 15, 25, and 35

sample_dict = functions.gather_data(
    location=(1, 1),
    sample_rate=51200,
    trigger_force=15,
    measurment_duration=0.3,
    num_presamples=100,
    hammer_trigger=False,
    resistance_trigger=5.4,
    R_shunt=350,
    num_samples=1,
    DIR_NAME="data/3x3_grid_30samples_3force",
    save_dict=False
)
functions.plot_data(sample_dict, sample_rate=51200)

## 4. Open and Visualize Saved Data

In [ ]:

sample_dict_2 = functions.open_pkl_dict(file_path="data/sampling_with_finger_location/0.6_0.8_17.3_7.pkl")
functions.plot_data(sample_dict_2)

In [ ]:
example = functions.open_pkl_dict(file_path="data/LHC_sampling_10samples/0.2_0.7_15_10.pkl")
functions.plot_data(example)
print(example["data"][0].shape)
print(example["hammer_max_force"])

In [ ]:
example["data"][0].shape[0]
len(example["data"][0][:, 0])

## 5. Latin Hypercube Sampling (LHC)

In [ ]:
hypercube_data = functions.generate_latin_hypercube_points(
    n_samples=40,
    x_range=(0.5, 1.0),
    y_range=(0, 1.0),
    seed=43,
    resolution=0.1
)
hypercube_data

In [ ]:
functions.run_random_sampling(
    sample_points=hypercube_data,
    sample_rate=51200,
    measurment_duration=0.3,
    num_samples=10,
    num_presamples=100, 
    hammer_trigger=True,
    trigger_force=10,
    resistance_trigger=1.942,
    R_shunt=350,
    save_dir_name="data/LHC_targeted_sampling_1_4_kvadrant",
    save_dict=True
)

## 6. Live Inference

In [ ]:
# Model definition
model = functions.ImpactPredictor(
    num_sensors=4,
    sequence_length=13,       # must match your down-sampled length
    cnn_filters=[32, 64, 128, 256],
    kernel_size=5,
    pool_size=2,
    force_reg=False,
    loc_class=True,
    grid_resolution=(3,3),
    latent_dim=256,
    dropout_rate=0.0,
    head_hidden_dim=256,
    head_hidden_layers=5
)

# Filter function and parameters
filter_fn = functions.filter_array_LI
filter_params = [
    900.0,   # cutoff frequency in Hz
    0.12     # trim duration in seconds
]

# Live inference
results = functions.live_inference(
    model=model,
    weight_path="model_weights/model_017_v22c_loc_c.pkl",
    board_size_xy=(0.5, 0.5),
    location=(1, 1),
    class_boundaries=(20, 30),
    grid_res=(3,3),
    sample_rate=51200,
    trigger_force=15,
    measurement_duration=0.3,
    num_presamples=100,
    hammer_trigger=True,
    resistance_trigger=5.5,
    DIR_NAME="live_inference",
    R_shunt=350,
    filter_parameters=filter_params,
    filter_fn=filter_fn,
    skip_data=60
)

# Inspect the returned dict
print(results)

In [ ]:
# Save live inference dict
functions.save_model_results(
    dir_name="live_inference",
    file_name="LI_1_1",
    model_results_dict=results
)  # saves a dictionary as a .pkl file

In [ ]:
# Open saved dicts
results_saved = functions.open_pkl_dict(file_path="live_inference/LI_0.5_0.5.pkl")

## References
- This code is part of: Enej Podlipnik, "Impact Location and Force Prediction," Undergraduate Diploma Thesis, University of Ljubljana, Faculty of Mechanical Engineering, 2024.
- Repository: [add your repository URL here]